In [ ]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
pip install nvidia-ml-py3

  Preparing metadata (setup.py) ... done
  Created wheel for nvidia-ml-py3: filename=nvidia_ml_py3-7.352.0-py3-none-any.whl size=19171 sha256=dfb31f48b447e3760112687faade8448cbac4bdf3ae1163cb4a918c2cd6335cd
  Stored in directory: /root/.cache/pip/wheels/5c/d8/c0/46899f8be7a75a2ffd197a23c8797700ea858b9b34819fbf9e
Successfully built nvidia-ml-py3


In [ ]:
pip install datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00


In [ ]:
pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.0 MB/s eta 0:00:00


In [ ]:
pip install transformers[torch]

In [ ]:
from pynvml import *


def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()

In [ ]:
# print_gpu_utilization()

In [ ]:

answers = pd.read_csv("drive/MyDrive/Levy/english_texts.csv")

In [ ]:
# Selecting only 'TEXT' and 'cEXT' columns and renaming them
df_ext = answers[['TEXT', 'cEXT']].rename(columns={'TEXT': 'text', 'cEXT': 'labels'})

# Reordering columns as per your requirement
df_ext = df_ext[['text', 'labels']]

In [ ]:
df_ext

,text,labels
0,"Well, right now I just woke up from a mid-day ...",0
1,"Well, here we go with the stream of consciousn...",0
2,An open keyboard and buttons to push. The thin...,0
3,I can't believe it! It's really happening! M...,1
4,"Well, here I go with the good old stream of co...",1
...,...,...
2958,I am motivated on a day to day basis by the ne...,1
2959,My son is the biggest part of my life and with...,1
2960,My kids and grandkids are what keeps me motiva...,1
2961,My biggest drive is to earn money so I can ret...,0


In [ ]:
from sklearn.model_selection import train_test_split

# Assuming df_ext is your DataFrame with 'text' and 'labels' columns

# Splitting the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    df_ext['text'], df_ext['labels'], random_state=42, test_size=0.2
)

# Creating a list of dictionaries for the test set
test_data = [{'label': int(label), 'text': text} for label, text in zip(y_test, X_test)]
train_data = [{'label': int(label), 'text': text} for label, text in zip(y_train, X_train)]

# Creating the final data structure
extraversion = {
    'train': train_data,
    'test': test_data
}


In [ ]:
len(extraversion['train'][0]['text'])

3927

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased", use_fast=True)

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [ ]:
import torch

def tokenize_and_chunk(texts, labels, tokenizer, chunk_size=510):
    input_id_chunks = []
    mask_chunks = []
    chunked_labels = []

    for text, label in zip(texts, labels):
        # Tokenize without special tokens
        tokens = tokenizer.encode_plus(text, add_special_tokens=False,
                                       return_tensors='pt', truncation=False)

        # Split into chunks of `chunk_size`
        input_ids = tokens['input_ids'][0]
        attention_mask = tokens['attention_mask'][0]
        for i in range(0, len(input_ids), chunk_size):
            input_ids_chunk = input_ids[i:i+chunk_size]
            attention_mask_chunk = attention_mask[i:i+chunk_size]

            # Add [CLS] and [SEP] tokens
            input_ids_chunk = torch.tensor([101] + input_ids_chunk.tolist() + [102])
            attention_mask_chunk = torch.tensor([1] + attention_mask_chunk.tolist() + [1])

            # Pad the sequences
            padding_length = 512 - len(input_ids_chunk)
            input_ids_chunk = torch.cat([input_ids_chunk, torch.zeros(padding_length, dtype=torch.long)])
            attention_mask_chunk = torch.cat([attention_mask_chunk, torch.zeros(padding_length, dtype=torch.long)])

            # Store the chunks
            input_id_chunks.append(input_ids_chunk)
            mask_chunks.append(attention_mask_chunk)
            chunked_labels.append(label)

    # Convert lists to tensors
    input_ids_tensor = torch.stack(input_id_chunks)
    attention_mask_tensor = torch.stack(mask_chunks)

    return input_ids_tensor, attention_mask_tensor, chunked_labels

# Training data
train_texts = [item['text'] for item in extraversion['train']]
train_labels = [item['label'] for item in extraversion['train']]
input_ids_tensor_train, attention_mask_tensor_train, chunked_train_labels = tokenize_and_chunk(train_texts, train_labels, tokenizer)

# Preparing the training dictionary
input_dict_train = {
    'input_ids': input_ids_tensor_train.long(),
    'attention_mask': attention_mask_tensor_train.int()
}

# Testing data
test_texts = [item['text'] for item in extraversion['test']]
test_labels = [item['label'] for item in extraversion['test']]
input_ids_tensor_test, attention_mask_tensor_test, chunked_test_labels = tokenize_and_chunk(test_texts, test_labels, tokenizer)

# Preparing the testing dictionary
input_dict_test = {
    'input_ids': input_ids_tensor_test.long(),
    'attention_mask': attention_mask_tensor_test.int()
}


Token indices sequence length is longer than the specified maximum sequence length for this model (865 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
import pandas as pd
from datasets import Dataset

# Convert tensors to lists (for DataFrame compatibility)
input_ids_list_train = input_ids_tensor_train.numpy().tolist()
attention_mask_list_train = attention_mask_tensor_train.numpy().tolist()

input_ids_list_test = input_ids_tensor_test.numpy().tolist()
attention_mask_list_test = attention_mask_tensor_test.numpy().tolist()

# Create DataFrame
df_train = pd.DataFrame({
    'input_ids': input_ids_list_train,
    'attention_mask': attention_mask_list_train,
    'labels': chunked_train_labels
})

df_test = pd.DataFrame({
    'input_ids': input_ids_list_test,
    'attention_mask': attention_mask_list_test,
    'labels': chunked_test_labels
})

# Create Hugging Face Dataset
train_dataset = Dataset.from_pandas(df_train)
test_dataset = Dataset.from_pandas(df_test)


In [ ]:
train_dataset['input_ids'][1]

[101,
 119,
 146,
 1341,
 146,
 1336,
 2215,
 1303,
 1155,
 1285,
 146,
 1176,
 1833,
 1250,
 1105,
 1833,
 1250,
 1218,
 146,
 1821,
 1280,
 1106,
 1202,
 1155,
 1104,
 1139,
 3203,
 3075,
 1104,
 1159,
 1177,
 1115,
 146,
 1336,
 1129,
 3075,
 1104,
 1155,
 1139,
 3553,
 1105,
 2789,
 1107,
 3754,
 5769,
 119,
 146,
 1567,
 1278,
 1184,
 1950,
 1202,
 146,
 1444,
 1106,
 15225,
 136,
 146,
 1138,
 1106,
 2951,
 1146,
 1111,
 1199,
 7857,
 1112,
 1218,
 146,
 112,
 173,
 1436,
 1202,
 1115,
 1145,
 119,
 146,
 1821,
 2296,
 1177,
 6600,
 1714,
 1268,
 1208,
 117,
 1304,
 8000,
 119,
 146,
 1341,
 146,
 1138,
 1724,
 1536,
 1133,
 1821,
 1136,
 1612,
 119,
 1177,
 4050,
 146,
 1821,
 1103,
 4459,
 2360,
 1113,
 1142,
 1362,
 1105,
 1209,
 8011,
 3708,
 1142,
 1362,
 1107,
 1199,
 1236,
 1107,
 1139,
 7218,
 119,
 1142,
 1110,
 1139,
 2273,
 1106,
 1849,
 1103,
 1362,
 1106,
 8011,
 3708,
 1122,
 1106,
 1129,
 1227,
 1106,
 1129,
 1103,
 1436,
 1120,
 1917,
 1106,
 3858,
 1106,
 1129,
 

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
import numpy as np
import evaluate

accuracy_metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"]
    precision = precision_metric.compute(predictions=predictions, references=labels)["precision"]
    recall = recall_metric.compute(predictions=predictions, references=labels)["recall"]
    f1score = f1_metric.compute(predictions=predictions, references=labels)["f1"]
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1score": f1score}

In [ ]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [ ]:
from transformers import BertConfig

# Initializing a RobertaConfig

configuration = BertConfig.from_pretrained(
    "bert-base-cased",
    num_labels=2,
    id2label=id2label,
    label2id=label2id
)

In [ ]:
configuration

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "NEGATIVE",
    "1": "POSITIVE"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "NEGATIVE": 0,
    "POSITIVE": 1
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.35.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

In [ ]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(
    "bert-base-cased",
    config=configuration
)


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# output_dir = '/content/drive/MyDrive/Levy/hyper_param_ext'


In [ ]:
from transformers import TrainingArguments, Trainer, logging

logging.set_verbosity_error()

In [ ]:
# print_gpu_utilization()

In [ ]:
# training_args = TrainingArguments(
#     output_dir=output_dir,
#     num_train_epochs=5,
#     weight_decay=0.01,
#     logging_steps = 4,
#     disable_tqdm=False,
#     evaluation_strategy="epoch",
#     save_strategy="epoch",
#     load_best_model_at_end=True,
#     push_to_hub=False
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_train,
#     eval_dataset=tokenized_test,
#     tokenizer=tokenizer,
#     data_collator=data_collator,
#     compute_metrics=compute_metrics,
# )

# result = trainer.train()
# print_summary(result)

In [ ]:
# trainer.evaluate()

In [ ]:
# print_gpu_utilization()


Hyperparamater tuning

In [ ]:
output_dir_hyper = '/content/drive/MyDrive/Levy/hyper_param_ext'


In [ ]:
%pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 10.2 MB/s eta 0:00:00


In [ ]:
def wandb_hp_space(trial):
    return {
        "method": "bayes",
        "metric": {"name": "objective", "goal": "minimize"},
        "parameters": {
            "learning_rate": {"distribution": "uniform", "min": 1e-6, "max": 5e-5},
            "per_device_train_batch_size": {"values": [4, 8]},
            "num_train_epochs": {"values": [5, 10]},
        },
    }

In [ ]:
def model_init():
    return BertForSequenceClassification.from_pretrained(
    "bert-base-cased", num_labels=2, id2label=id2label, label2id=label2id
)

In [ ]:
training_args_hyper = TrainingArguments(
    output_dir=output_dir_hyper,
    per_device_eval_batch_size=16,
    per_device_train_batch_size=16,
    weight_decay=0.01,
    logging_steps = 4,
    disable_tqdm=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    gradient_accumulation_steps=8,
    gradient_checkpointing=True,
    fp16=True,
)

In [ ]:
trainer = Trainer(
    model=None,
    model_init=model_init,
    args=training_args_hyper,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

In [ ]:
best_trial = trainer.hyperparameter_search(
    direction="minimize",
    backend="wandb",
    hp_space=wandb_hp_space,
    n_trials=15,
)

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: g4i6ygbr
Sweep URL: https://wandb.ai/jads/uncategorized/sweeps/g4i6ygbr


...

wandb: WARNING Ignored wandb.init() arg project when running a sweep.


wandb: WARNING Config item 'per_device_train_batch_size' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'num_train_epochs' was locked by 'sweep' (ignored update).
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1score
0,0.701100,0.697878,0.511749,0.525253,0.610738,0.564779
1,0.696500,0.695504,0.529156,0.534678,0.711409,0.610511
2,0.694100,0.692985,0.539600,0.536138,0.833893,0.652659
3,0.690500,0.691658,0.532637,0.530633,0.857383,0.655548
4,0.689800,0.690484,0.539600,0.553090,0.585570,0.568867
5,0.686800,0.688287,0.549173,0.553571,0.676174,0.608761
6,0.684400,0.688384,0.545692,0.543529,0.775168,0.639004
8,0.676500,0.688075,0.543951,0.547745,0.692953,0.611852
9,0.673000,0.688532,0.550914,0.551282,0.721477,0.625000


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

eval/accuracy,▁▄▆▅▆█▇▆▇█
eval/f1score,▁▅██▁▄▇▄▅▆
eval/loss,█▆▅▄▃▁▂▁▁▂
eval/precision,▁▃▄▂██▆▆▇▇
eval/recall,▂▄▇█▁▃▆▃▄▅
eval/runtime,▅▁██▂▂▆▆▁▆
eval/samples_per_second,▄█▁▁▇▇▃▃█▃
eval/steps_per_second,▄█▁▁▇▇▃▃█▃
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train/learning_rate,███▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
